## Initialize

In [1]:
from dummy_spark import SparkContext
# from pyspark import SparkContext

In [2]:
from sympy import *
from drudge import *
from agp_fermi import *

In [3]:
ctx = SparkContext()

In [4]:
dr = AGPFermi(ctx)

In [5]:
names = dr.names

In [6]:
p, q, r, s  = names.A_dumms[:4]

## Creation / annihilation operators

In [7]:
c_p = dr.sum(names.c_[p, UP])

In [8]:
cdag_p = dr.sum(names.c_dag[p, DOWN])

In [9]:
cp2 = names.c_[p, UP]
type(cp2)

drudge.term.Vec

In [10]:
print(cp2.base)
print(cp2.indices)

c
(CranChar.AN, p, SpinOneHalf.UP)


In [11]:
(c_p*cdag_p + cdag_p*c_p).display()

<IPython.core.display.Math object>

In [12]:
dr.simplify(c_p*cdag_p + cdag_p*c_p).display()

<IPython.core.display.Math object>

In [13]:
(dr.fermi_dr.dagger(c_p)).display()

<IPython.core.display.Math object>

## Pairing operators

In [14]:
P_p = dr.sum( dr.P[p] )
Pdag_q = dr.sum( dr.Pdag[q] )

In [15]:
Pdag_q.display()

<IPython.core.display.Math object>

In [16]:
P_p.display()

<IPython.core.display.Math object>

In [17]:
expr = dr.simplify(P_p * Pdag_q - Pdag_q * P_p)
expr.display()

<IPython.core.display.Math object>

## Spin range sum

In [18]:
dr.sum(names.c_[p,UP]).display()

<IPython.core.display.Math object>

In [19]:
(dr.sum((s,dr.spin_range),names.c_[p,s])).simplify().display()

<IPython.core.display.Math object>

## Spin flip su(2)

In [20]:
Sp = dr.sum(names.J_p[p])
Sm = dr.sum(names.J_m[q])

In [21]:
Sp.display()

<IPython.core.display.Math object>

In [22]:
expr = dr.simplify( Sp * Sm - Sm * Sp )
expr.display()

<IPython.core.display.Math object>

## Relation between spin flip and bcs

In [23]:
expr2 = Sp * Pdag_q * Sm
expr2.display()

<IPython.core.display.Math object>

In [24]:
expr2.simplify().display()

<IPython.core.display.Math object>

## Unitary group operators

In [25]:
e_ = names.e_
expr = dr.simplify(e_[q,p])
expr.display()

<IPython.core.display.Math object>

## BCS with $c^\dagger$

In [26]:
expr = dr.simplify(cdag_p | Pdag_q)
expr.simplify().display()

<IPython.core.display.Math object>

In [27]:
cdag_p.display()

<IPython.core.display.Math object>

In [28]:
P_q = dr.sum(names.P_[q])
expr = dr.simplify(cdag_p | P_q)
expr.simplify().display()

<IPython.core.display.Math object>

In [29]:
cdag_pup = dr.sum(names.c_dag[p, UP])
expr = dr.simplify(cdag_pup | P_q)
expr.simplify().display()

<IPython.core.display.Math object>

In [30]:
c_pup = c_p
c_pdn = dr.sum(names.c_[p, DOWN])
expr = (c_pup | Pdag_q).simplify()
expr.display()

<IPython.core.display.Math object>

In [31]:
expr = (c_pdn | Pdag_q).simplify()
expr.display()

<IPython.core.display.Math object>

In [32]:
Nq = dr.sum(names.N_[q])
expr = (c_pup | Nq).simplify()
expr.display()

<IPython.core.display.Math object>

## SU2 with $c^\dagger$

In [33]:
Jm_q = dr.sum(names.J_m[q])
Jp_q = dr.sum(names.J_p[q])
Jz_q = dr.sum(names.J_z[q])

In [34]:
(cdag_pup | Jm_q).simplify().display()

<IPython.core.display.Math object>

In [35]:
(cdag_pup | Jz_q).simplify().display()

<IPython.core.display.Math object>

In [36]:
(c_pup | Jz_q).simplify().display()

<IPython.core.display.Math object>

## Extract SU2 and Pairing operators

In [37]:
epp = e_[p,p]

In [38]:
dr.simplify(epp).display()

<IPython.core.display.Math object>

In [39]:
num_op = dr.extract_su2(dr.sum(epp))
num_op.simplify().display()

<IPython.core.display.Math object>

In [40]:
# t = IndexedBase('t')
epq2 = dr.simplify(dr.sum(e_[p,q]*e_[p,q]))
epq2.display()

<IPython.core.display.Math object>

In [41]:
type(epq2)

drudge.drudge.Tensor

In [42]:
epq2 = dr.canon_indices(epq2)
epq2.display()

<IPython.core.display.Math object>

In [43]:
pdag_fermi = dr.simplify(names.c_dag[p, UP]*names.c_dag[p, DOWN])
dr.extract_su2(pdag_fermi).display()

<IPython.core.display.Math object>

In [44]:
p_fermi = dr.simplify(names.c_[p, DOWN] * names.c_[p, UP])
dr.extract_su2(p_fermi).display()

<IPython.core.display.Math object>

In [45]:
epq2_bcs = dr.extract_su2(epq2)
epq2_bcs.simplify().display()

<IPython.core.display.Math object>

In [46]:
sp_fermi = dr.simplify(names.c_dag[p, UP] * names.c_[p, DOWN])

In [47]:
sp_fermi.display()

<IPython.core.display.Math object>

In [48]:
dr.extract_su2(sp_fermi).display()

<IPython.core.display.Math object>

## Nil-potency

In [49]:
expr = cdag_p*cdag_p

In [50]:
expr.display()

<IPython.core.display.Math object>

In [51]:
expr.simplify().display()

<IPython.core.display.Math object>

## AGP-DPQ RPA

#### 1. Pairing Hamiltonian

In [52]:
N = names.N
Pdag = names.P_dag
P = names.P_

In [53]:
eps = IndexedBase('\epsilon')
G = Symbol('G')
H = dr.einst(eps[p]*N[p]) - G*dr.sum((p,names.A), (q, names.A), Pdag[p] * P[q])
Ham_fermi = dr.einst(eps[p]*e_[p,p]) \
    - G*dr.sum((p,names.A), (q, names.A), e_[p,q]*e_[p,q] - KroneckerDelta(p,q)*e_[p,q])/2

In [54]:
H = dr.simplify(H)
H.display()

<IPython.core.display.Math object>

In [55]:
Ham_fermi = dr.simplify(Ham_fermi)
Ham_fermi.display()

<IPython.core.display.Math object>

In [56]:
eta = IndexedBase('\eta')
D_pq = dr.sum(eta[p]*e_[p,q] - eta[q]*e_[q,p])
D_dag_pq = dr.sum(eta[p]*e_[q,p] - eta[q]*e_[p,q])

In [57]:
D_pq.display()

<IPython.core.display.Math object>

In [58]:
D_dag_pq.merge().display()

<IPython.core.display.Math object>

In [59]:
term1 = dr.simplify( Ham_fermi | D_dag_pq )
term1.display()

<IPython.core.display.Math object>

In [60]:
term2 = dr.simplify( D_pq | term1 )
term2.display()

<IPython.core.display.Math object>

In [61]:
# Amat_pq = dr.simplify((D_pq | (H | D_dag_pq)))
Amat_pq = term2
Amat_pq.display()

<IPython.core.display.Math object>

In [62]:
Amat_pq.merge().display()

<IPython.core.display.Math object>

In [63]:
ans = dr.get_seniority_zero(Amat_pq)
ans = dr.simplify(ans)
ans.merge().display()

<IPython.core.display.Math object>

-----

In [65]:
from agp_fermi import _even_fermi_filter, _get_fermi_partitions, _generate_partitions

In [66]:
Amat_pq.n_terms

95

In [67]:
exp1 = Amat_pq.filter(lambda x: _even_fermi_filter(x, spec=dr._spec))
exp1.n_terms

95

In [68]:
exp2 = dr.simplify( dr.simplify( dr.extract_su2(exp1) ) )
exp2.merge().display()

<IPython.core.display.Math object>

In [69]:
exp3 = exp2.bind(lambda x: _get_fermi_partitions(x, spec=dr._spec))
exp3 = dr.simplify(exp3)
exp3.merge().display()

<IPython.core.display.Math object>

In [70]:
exp4 = dr.simplify(exp3)
exp4 = dr.simplify(dr.simplify(dr.extract_su2(exp4)))
exp4.merge().display()

<IPython.core.display.Math object>